In [1]:
library(tsfeatures)
library(imputeTS)
library(forecast)
library(dplyr)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
data=read.csv('Imports.csv')

In [3]:
head(data)

,月份,日本,经合组织成员国,OECD成员国_欧洲,OECD和六个非成员经济体,主要亚洲五国.OECD.,瑞士,韩国,土耳其,比利时,⋯,中华人民共和国,南非,哥伦比亚,墨西哥,新西兰,以色列,哥斯达黎加,欧元区,俄罗斯,爱尔兰
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1995-01,100.86,100.98,100.79,100.34,100.74,99.21,NA,NA,100.50,⋯,101.44,101.91,NA,NA,101.35,NA,NA,100.62,NA,102.04
2,1995-02,100.60,100.85,100.73,100.26,100.78,98.84,NA,NA,100.42,⋯,101.95,101.79,NA,NA,101.32,NA,NA,100.56,NA,102.02
3,1995-03,100.34,100.71,100.69,100.15,100.80,98.80,NA,NA,100.46,⋯,102.36,101.73,NA,NA,101.38,NA,NA,100.57,NA,101.89
4,1995-04,100.14,100.66,100.75,100.11,100.79,98.97,NA,NA,100.47,⋯,102.63,101.73,NA,NA,101.55,NA,NA,100.67,NA,101.70
5,1995-05,100.05,100.69,100.85,100.12,100.77,99.18,NA,NA,100.46,⋯,102.72,101.78,NA,NA,101.77,NA,NA,100.78,NA,101.67
6,1995-06,100.04,100.78,100.97,100.18,100.73,99.33,NA,NA,100.44,⋯,102.63,101.83,NA,NA,101.94,NA,NA,100.80,NA,101.76


In [4]:
tail(data)

,月份,日本,经合组织成员国,OECD成员国_欧洲,OECD和六个非成员经济体,主要亚洲五国.OECD.,瑞士,韩国,土耳其,比利时,⋯,中华人民共和国,南非,哥伦比亚,墨西哥,新西兰,以色列,哥斯达黎加,欧元区,俄罗斯,爱尔兰
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
341,2023-05,98.40,98.30,98.63,97.17,95.11,96.91,99.64,99.75,99.31,⋯,93.33,96.46,95.95,103.91,96.67,97.84,101.81,98.16,99.61,NA
342,2023-06,98.53,98.49,98.73,97.21,94.89,96.99,100.02,98.74,99.40,⋯,92.96,96.54,96.52,104.36,96.79,97.71,102.00,98.39,NA,NA
343,2023-07,98.57,98.59,98.61,97.26,94.81,97.04,100.26,96.99,99.56,⋯,92.84,96.85,96.68,104.87,96.78,97.74,102.38,NA,NA,NA
344,2023-08,98.48,98.50,98.36,97.22,94.82,96.93,100.24,95.26,99.63,⋯,92.88,97.28,96.64,105.27,96.70,97.62,102.82,NA,NA,NA
345,2023-09,98.35,98.33,98.13,97.14,94.86,96.56,99.99,94.77,99.60,⋯,93.00,97.76,96.59,105.55,96.58,NA,NA,NA,NA,NA
346,2023-10,98.27,98.09,97.90,96.90,NA,96.02,99.69,94.95,99.55,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [5]:
data=data[1:285,c(2:dim(data)[2])]

In [25]:
dim(data)

[1] 345  44

In [7]:
which_baddata=function(data){
baddata=c()
for(i in 1:dim(data)[2])
{loc=min(which(is.na(data[,i])==FALSE))-1
if((dim(data)[1]-loc)<40)
    {
    baddata=append(baddata,i)
}
if(sum(is.na(data[loc:dim(data)[1],i])==TRUE)>10)
 {
     baddata=append(baddata,i)
 }
 if(sum(is.na(data[(dim(data)[1]-11):dim(data)[1],i])==TRUE)>3)
 {
     baddata=append(baddata,i)
 }
}
baddata=unique(baddata)
return(baddata)}

In [8]:
baddata=which_baddata(data)

In [9]:
baddata

[1] 45 46

In [10]:
data=data[,-1*baddata]

In [11]:
data

,日本,经合组织成员国,OECD成员国_欧洲,OECD和六个非成员经济体,主要亚洲五国.OECD.,瑞士,韩国,土耳其,比利时,丹麦,⋯,西方七国,印尼,中华人民共和国,南非,哥伦比亚,墨西哥,新西兰,以色列,哥斯达黎加,欧元区
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,100.86,100.98,100.79,100.34,100.74,99.21,NA,NA,100.50,101.19,⋯,100.79,NA,101.44,101.91,NA,NA,101.35,NA,NA,100.62
2,100.60,100.85,100.73,100.26,100.78,98.84,NA,NA,100.42,101.24,⋯,100.67,NA,101.95,101.79,NA,NA,101.32,NA,NA,100.56
3,100.34,100.71,100.69,100.15,100.80,98.80,NA,NA,100.46,101.22,⋯,100.51,NA,102.36,101.73,NA,NA,101.38,NA,NA,100.57
4,100.14,100.66,100.75,100.11,100.79,98.97,NA,NA,100.47,101.16,⋯,100.46,NA,102.63,101.73,NA,NA,101.55,NA,NA,100.67
5,100.05,100.69,100.85,100.12,100.77,99.18,NA,NA,100.46,101.09,⋯,100.49,NA,102.72,101.78,NA,NA,101.77,NA,NA,100.78
6,100.04,100.78,100.97,100.18,100.73,99.33,NA,NA,100.44,101.01,⋯,100.60,NA,102.63,101.83,NA,NA,101.94,NA,NA,100.80
7,100.10,100.86,101.00,100.21,100.68,99.41,NA,NA,100.27,101.03,⋯,100.69,NA,102.40,101.89,NA,NA,102.02,NA,NA,100.72
8,100.24,100.83,100.94,100.17,100.65,99.46,NA,NA,99.91,101.13,⋯,100.66,NA,102.14,101.95,NA,NA,102.01,NA,NA,100.56
9,100.41,100.70,100.80,100.03,100.66,99.47,NA,NA,99.50,101.18,⋯,100.50,NA,101.93,102.03,NA,NA,101.93,NA,NA,100.34


In [52]:
freq=12
ll=dim(data)[2]
hh=12
start=c(2000,1)
m=5
n=4

In [53]:
process_data <- function(data, start, freq, hh, ll) {
  data_ts <- list()
  data_tstrain <- list()
  data_features <- NULL
  
  for (i in 1:ll) {
    # 寻找第一个非NA值的位置
    loc <- min(which(is.na(data[, i]) == FALSE)) - 1
    zc <- loc %/% freq
    yu <- loc %% freq
    
    # 提取子序列
    x <- data[(loc + 1):dim(data)[1], i]
    
    # 检测并处理NA值
    r <- which(is.na(x) == TRUE)
    if (length(r) != 0) {
      x <- na_kalman(x) # 假设na_kalman是一个处理NA值的函数
    }
    
    # 创建时间序列对象
    da <- ts(x[1:(length(x) - hh)], start = c(start[1] + zc, start[2] + yu), frequency = freq)
    da1 <- ts(x, start = c(start[1] + zc, start[2] + yu), frequency = freq)
    
    # 保存时间序列对象
    data_ts[[i]] <- da1
    data_tstrain[[i]] <- da
    
    # 提取时间序列特征
    f <- tsfeatures(da)
    if (is.null(data_features)) {
      data_features <- f
    } else {
      data_features <- rbind(data_features, f)
    }
  }
  
  # 函数返回一个列表，包含处理后的时间序列和特征
  list(time_series = data_ts, time_series_train = data_tstrain, features = data_features)
}


In [54]:
processed_data <- process_data(data, start = c(1995, 1), freq, hh, ll)

In [55]:
data_ts=processed_data$time_series
data_tstrain=processed_data$time_series_train
data_features=processed_data$features

In [56]:
head(data_features)

frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12,1,12,0.9414541,1.469496e-07,-4.148065,-3.765981,0.8454210,1.159314,0.12604369,12,5,0.5977804,0.9774598,5.517595,0.8238031,0.9675078,0.5318010,0.9109732,0.3526531
12,1,12,0.9630601,8.844036e-08,-8.774571,2.937463,0.8582569,1.183075,0.09641165,1,5,0.5427694,0.9757847,6.052019,0.8190963,0.9426224,0.4867796,0.6482644,0.5042263
12,1,12,0.9656192,5.579385e-08,-8.151818,1.956549,0.8807846,1.410732,0.10045470,9,4,0.5458390,0.9712250,5.603514,0.8506849,1.2190287,0.4892918,0.5528145,0.4555680
12,1,12,0.9531315,1.222234e-07,-3.941638,1.944559,0.8604031,1.131996,0.24430725,1,5,0.5882649,0.9620671,4.573657,0.8462332,1.0444014,0.5709416,0.8613847,0.3172945
12,1,12,0.9492339,1.599164e-07,3.179791,5.670570,0.8527939,1.177701,0.39023178,1,6,0.6173344,0.9634733,4.700404,0.8277382,1.0134493,0.5072502,0.6229488,0.4188087
12,1,12,0.9320480,1.708380e-07,-2.018620,-6.632152,0.8763513,1.222079,0.25096789,11,5,0.6383962,0.9543605,3.877693,0.8660764,1.1364421,0.6677847,0.9972327,0.1575627


In [57]:
length=matrix(0,ll,1)
for( i in 1:ll)
    {
    length[i,1]=length(data_tstrain[[i]])
}

In [58]:
data_features=cbind(data_features,length)

In [59]:
head(data_features)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,12,1,12,0.9414541,1.469496e-07,-4.148065,-3.765981,0.8454210,1.159314,0.12604369,⋯,5,0.5977804,0.9774598,5.517595,0.8238031,0.9675078,0.5318010,0.9109732,0.3526531,333
2,12,1,12,0.9630601,8.844036e-08,-8.774571,2.937463,0.8582569,1.183075,0.09641165,⋯,5,0.5427694,0.9757847,6.052019,0.8190963,0.9426224,0.4867796,0.6482644,0.5042263,333
3,12,1,12,0.9656192,5.579385e-08,-8.151818,1.956549,0.8807846,1.410732,0.10045470,⋯,4,0.5458390,0.9712250,5.603514,0.8506849,1.2190287,0.4892918,0.5528145,0.4555680,333
4,12,1,12,0.9531315,1.222234e-07,-3.941638,1.944559,0.8604031,1.131996,0.24430725,⋯,5,0.5882649,0.9620671,4.573657,0.8462332,1.0444014,0.5709416,0.8613847,0.3172945,333
5,12,1,12,0.9492339,1.599164e-07,3.179791,5.670570,0.8527939,1.177701,0.39023178,⋯,6,0.6173344,0.9634733,4.700404,0.8277382,1.0134493,0.5072502,0.6229488,0.4188087,333
6,12,1,12,0.9320480,1.708380e-07,-2.018620,-6.632152,0.8763513,1.222079,0.25096789,⋯,5,0.6383962,0.9543605,3.877693,0.8660764,1.1364421,0.6677847,0.9972327,0.1575627,333


In [20]:
f1=function(x){
    if( x%%1>=0.5)
        {
        return(as.integer(x)+1)
    }
    else{return(as.integer(x))}
}

In [21]:
forecast_and_evaluate <- function(data_ts, data_tstrain, m,n,hh, ll) {
  predets <- array(0, dim = c(ll, m, n, 6))
  predhets <- array(0, dim = c(ll, m, n, hh))
  predthetaf <- array(0, dim = c(ll, m, n, 6))
  predhthetaf <- array(0, dim = c(ll, m, n, hh))
  yreal <- list()
  
  for (k in 1:ll) {
    y <- data_tstrain[[k]]
    yall <- data_ts[[k]]
    y_pred <- ts(yall[(length(yall) - hh + 1):length(yall)], end = end(yall), frequency = frequency(yall))
    y_l=length(y)
    loc = 1:length(y)
    loc_m = as.integer(loc*m/length(y))
    filt_d = data.frame(y,loc_m)
    filt_0=filter(filt_d ,loc_m==0)
    m_l=count(filt_0)
    n_l=m_l/n
    m_l=as.integer(m_l)
    n_l=as.numeric(n_l)
    
    yreal[[k]] <- y_pred
    
    for (i in 0:(m - 1)) {
      for (j in 0:(n-1)) {
        Y <- ts(y[round((i*m_l+j*n_l)+1):y_l], end = end(y), frequency = frequency(yall))
        M <- ets(Y)
        pd <- forecast(M, h = hh)
        predhets[k, (i + 1), (j + 1), ] <- pd$mean
        predets[k, (i + 1), (j + 1), ] <- accuracy(pd, y_pred)[2, 1:6]
        
        M <- thetaf(Y, h = hh)
        pd <- forecast(M, h = hh)
        predhthetaf[k, (i + 1), (j + 1), ] <- pd$mean
        predthetaf[k, (i + 1), (j + 1), ] <- accuracy(pd, y_pred)[2, 1:6]
      }
    }
  }
  
  list(
    pred_ets = list(predhets = predhets, predets = predets),
    pred_thetaf = list(predhthetaf = predhthetaf, predthetaf = predthetaf),
    y_real = yreal
  )
}

In [22]:
forecast_list=forecast_and_evaluate(data_ts, data_tstrain, m,n,hh, ll)

In [23]:
predhets=forecast_list$pred_ets$predhets
predets=forecast_list$pred_ets$predets
predhthetaf=forecast_list$pred_thetaf$predhthetaf
predthetaf=forecast_list$pred_thetaf$predthetaf
y_real=forecast_list$y_real

In [24]:
length5=matrix(0,ll,m)
for(k in 1:ll)
{
y = data_tstrain[[k]]
y_l=length(y)
loc = 1:length(y)
loc_m = as.integer(loc*m/length(y))
filt_d = data.frame(y,loc_m)
filt_0=filter(filt_d ,loc_m==0)
m_l=count(filt_0)
m_l=as.integer(m_l)
length5[k,]=c(0,1*m_l,2*m_l,3*m_l,4*m_l)
    }

### 进行数据模拟

### 训练数据特征提取

In [26]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library(lightgbm)


Attaching package: ‘xgboost’


The following object is masked from ‘package:dplyr’:

    slice



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice


The following object is masked from ‘package:dplyr’:

    slice




In [28]:
library(lightgbm)

In [47]:
etsxgbreg <- readRDS("etsxgbreg.rds")
etsxgbcls <- readRDS("etsxgbcls.rds")
etslgbreg <- readRDS("etslgbreg.rds")
etslgbcls <- readRDS("etslgbcls.rds")

In [48]:
thetafxgbreg <- readRDS("thetafxgbreg.rds")
thetafxgbcls <- readRDS("thetafxgbcls.rds")
thetaflgbreg <- readRDS("thetaflgbreg.rds")
thetaflgbcls <- readRDS("thetaflgbcls.rds")

In [38]:
meanunique=function(x)
    {
    mean(unique(x))
}

In [39]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

In [40]:
realbest_construct<-function(MASE)
    {
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    line=apply(count,1,meanunique)
    if (max(line,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        line[is.na(line)]=100 
        realbestmean[i,]= whichmin(line)
    }
    }
    return(realbestmean)
    }
    

In [49]:
etslgbreg

LightGBM Model
(Booster handle is invalid)

In [45]:
etsxgbcls

##### xgb.Booster
raw: 1.5 Mb 
call:
  xgb.train(params = params, data = dtrain, nrounds = nrounds, 
    watchlist = watchlist, verbose = verbose, print_every_n = print_every_n, 
    early_stopping_rounds = early_stopping_rounds, maximize = maximize, 
    save_period = save_period, save_name = save_name, xgb_model = xgb_model, 
    callbacks = callbacks, objective = "multi:softmax", num_class = 5)
params (as set within xgb.train):
  objective = "multi:softmax", num_class = "5", silent = "1"
xgb.attributes:
  niter
callbacks:
  cb.print.evaluation(period = print_every_n)
  cb.evaluation.log()
  cb.save.model(save_period = save_period, save_name = save_name)
niter: 100
evaluation_log:
     iter train_merror
    <num>        <num>
        1     0.561750
        2     0.552521
---                   
       99     0.326021
      100     0.324354

## 预测与输出误差

In [60]:
alldatalgb <- lgb.Dataset(data = as.matrix(data_features))
alldataxgb <- xgb.DMatrix(data = as.matrix(data_features))

In [61]:
xgbregets=predict(etsxgbreg,alldataxgb)
xgbclsets=predict(etsxgbcls,alldataxgb)
lgbregets=predict(etslgbreg,as.matrix(data_features))
lgbclsets=predict(etslgbcls,as.matrix(data_features))

In [62]:
xgbregthetaf=predict(thetafxgbreg,alldataxgb)
xgbclsthetaf=predict(thetafxgbcls,alldataxgb)
lgbregthetaf=predict(thetaflgbreg,as.matrix(data_features))
lgbclsthetaf=predict(thetaflgbcls,as.matrix(data_features))

In [63]:
lgbclsm=matrix(lgbclsets,5,ll)
lgbclsetsr=matrix(0,ll,1)
for(i in 1:ll)
    {
    lgbclsetsr[i,]=which.max(lgbclsm[,i])
}

In [64]:
lgbclsm=matrix(lgbclsthetaf,5,ll)
lgbclsthetafr=matrix(0,ll,1)
for(i in 1:ll)
    {
    lgbclsthetafr[i,]=which.max(lgbclsm[,i])
}

In [65]:
round10=function(x)
    {
    l=round(x)
    for(i in 1:length(l))
    {
    if(l[i]>5)
        {
        l[i]=5
    }
    if(l[i]<1)
    {
        l[i]=1
    }
}
    l
}

In [66]:
ets_pr=matrix(0,ll,4)
ets_pr[,1]=round10(xgbregets+1)
ets_pr[,2]=xgbclsets+1
ets_pr[,3]=round10(lgbregets+1)
ets_pr[,4]=lgbclsetsr

In [67]:
thetaf_pr=matrix(0,ll,4)
thetaf_pr[,1]=round10(xgbregthetaf+1)
thetaf_pr[,2]=xgbclsthetaf+1
thetaf_pr[,3]=round10(lgbregthetaf+1)
thetaf_pr[,4]=lgbclsthetafr

In [68]:
meanpre=function(pred_array,opt_pr,h,testindex)
    {
    data_length=length(testindex)
    meanpreh=matrix(0,data_length,h)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    predh=pred_array[index,loc,,]
    predh=unique(predh)
    meanpreh[i,]=apply(predh,2,mean)
}
    return(meanpreh)
}

In [69]:
predres=function(pred_matrix,data,real_data,opt_pr,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]])
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]])
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    start=lengthmatrix[i,loc]
    y_all=data[[index]]
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=real_data[[index]]
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [70]:
testindex=c(1:ll)
lengthtest=length5[testindex,]
etsxgbcls=meanpre(predhets,ets_pr[,1],hh,testindex)
etsxgbreg=meanpre(predhets,ets_pr[,2],hh,testindex)
etslgbcls=meanpre(predhets,ets_pr[,3],hh,testindex)
etslgbreg=meanpre(predhets,ets_pr[,4],hh,testindex)
etsxgbclsres=predres(etsxgbcls,data_tstrain,y_real,ets_pr[,1],lengthtest,hh,testindex)
etsxgbregres=predres(etsxgbreg,data_tstrain,y_real,ets_pr[,2],lengthtest,hh,testindex)
etslgbclsres=predres(etslgbcls,data_tstrain,y_real,ets_pr[,3],lengthtest,hh,testindex)
etslgbregres=predres(etslgbreg,data_tstrain,y_real,ets_pr[,4],lengthtest,hh,testindex)

In [71]:
thetafxgbcls=meanpre(predhthetaf,thetaf_pr[,1],hh,testindex)
thetafxgbreg=meanpre(predhthetaf,thetaf_pr[,2],hh,testindex)
thetaflgbcls=meanpre(predhthetaf,thetaf_pr[,3],hh,testindex)
thetaflgbreg=meanpre(predhthetaf,thetaf_pr[,4],hh,testindex)
thetafxgbclsres=predres(thetafxgbcls,data_tstrain,y_real,thetaf_pr[,1],lengthtest,hh,testindex)
thetafxgbregres=predres(thetafxgbreg,data_tstrain,y_real,thetaf_pr[,2],lengthtest,hh,testindex)
thetaflgbclsres=predres(thetaflgbcls,data_tstrain,y_real,thetaf_pr[,3],lengthtest,hh,testindex)
thetaflgbregres=predres(thetaflgbreg,data_tstrain,y_real,thetaf_pr[,4],lengthtest,hh,testindex)

In [72]:
apply(predets[,1,1,],2,mean)
apply(etsxgbclsres,2,mean)
apply(etsxgbregres,2,mean)
apply(etslgbclsres,2,mean)
apply(etslgbregres,2,mean)

[1] 1.724627 2.212382 1.957473 1.758364 1.998536 1.565424

[1] 1.691344 2.318982 2.040764 1.726878 2.085029 1.516860

[1] 1.731742 2.314214 2.032916 1.767660 2.077611 1.545274

[1] 1.696022 2.290020 2.014204 1.731155 2.058475 1.508934

[1] 1.814732 2.375999 2.092874 1.852679 2.138639 1.504679

In [73]:
apply(predthetaf[,1,1,],2,mean)
apply(thetafxgbclsres,2,mean)
apply(thetafxgbregres,2,mean)
apply(thetaflgbclsres,2,mean)
apply(thetaflgbregres,2,mean)

[1] 1.388740 1.743146 1.529509 1.410952 1.557603 1.186096

[1] 1.375818 1.715313 1.500155 1.397946 1.527634 1.078131

[1] 1.394537 1.750928 1.532373 1.416963 1.560560 1.171046

[1] 1.384497 1.723400 1.507957 1.406702 1.535572 1.111834

[1] 1.465489 1.804156 1.584095 1.489662 1.613414 1.123010

In [75]:
alltest=matrix(0,10,6)
alltest[1,]=apply(predets[,1,1,],2,mean)
alltest[2,]=apply(etsxgbclsres,2,mean)
alltest[3,]=apply(etsxgbregres,2,mean)
alltest[4,]=apply(etslgbclsres,2,mean)
alltest[5,]=apply(etslgbregres,2,mean)

alltest[6,]=apply(predthetaf[,1,1,],2,mean)
alltest[7,]=apply(thetafxgbclsres,2,mean)
alltest[8,]=apply(thetafxgbregres,2,mean)
alltest[9,]=apply(thetaflgbclsres,2,mean)
alltest[10,]=apply(thetaflgbregres,2,mean)

In [77]:
write.csv(alltest,'Imports_M4_12.csv')